Importing libraries

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as condition
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from openpyxl import load_workbook

Initiating all the variables

In [2]:
dnb_financeanalytics = 'https://na4.dnbi.com/dnbiweb/#/dnbi/showQuickSearchResult'
waiting_time = 60
dnb_username = 'username'
dnb_password = 'password'
directory_sheet_name = 'Scraped Data'
directory_path = r"C:\Users\Owner\PY4E\DnB Webscraping\China_Household Appliances19-19.xlsx"

Reading the business directory file 

In [3]:
# We rely on data from DnB Business Directory to initiate scraping in DnB Finance Analytics
with pd.ExcelFile(directory_path) as xls:
    file = pd.read_excel(xls, sheet_name=directory_sheet_name)
    Business_Directory_DF = pd.DataFrame(file)

Business_Directory_DF.head(5)

,company,directory_link,address,state,country,website,first3_address
0,"Shenzhen Angell Medical Equipment Co., Ltd.",https://www.dnb.com/business-directory/company...,"West of 2/F, Building 5, Nanshui Industrial Vi...",Guangdong,China,http://www.comermy.com.cn/,"West of 2/F,"
1,Jiangsu Huadong Medical Instrument Industrial ...,https://www.dnb.com/business-directory/company...,"No.88 Tongda Road, Touqiao Town, Hanjiang Dist...",Yangzhou,China,http://www.yadamed.com/,"No.88 Tongda Road,"
2,"Tianjin Zhite Medical Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"East Zhang Gui Zhuang Railroad, Wai Huan Road,...",Tianjin,China,NaN,East Zhang Gui
3,"Ningbo Cibei Medical Applicances Co.,Ltd",https://www.dnb.com/business-directory/company...,"No.8, Xiaotuanpu E. Road, Dong Development Zon...",Cixi,China,http://www.cbzj.com.cn/,"No.8, Xiaotuanpu E."
4,"Guilin Weirun Medical Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"No.D-07, Informationa Industrial Park, High-Te...",Guilin,China,http://www.veirun.com/,"No.D-07, Informationa Industrial"


Opening the browser and signing in to DnB Finance Analytics

In [4]:
# Opening Google Chrome
driver = webdriver.Chrome()

# Opening DnB Finance Analytics
driver.get(dnb_financeanalytics)
wait = WebDriverWait(driver, waiting_time)
wait.until(
    condition.element_to_be_clickable((By.XPATH, '//*[@id="idp-discovery-submit"]'))
)

# Filling out the username
username = driver.find_element(By.XPATH, '//*[@id="idp-discovery-username"]')
username.clear()
username.send_keys(dnb_username)

# Submit the username
submit_username = driver.find_element(By.XPATH, '//*[@id="idp-discovery-submit"]')
submit_username.click()

# Filling out the password
wait.until(
    condition.presence_of_element_located((By.XPATH, '//*[@id="okta-signin-password"]'))
)
password = driver.find_element(By.XPATH, '//*[@id="okta-signin-password"]')
password.clear()
password.send_keys(dnb_password)

# Submit username and password
submit_login = driver.find_element(By.XPATH, '//*[@id="okta-signin-submit"]')
submit_login.click()

# Waiting for the Finance Analytics Dashboard
wait.until(
    condition.element_to_be_clickable(
        (
            By.XPATH,
            '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/app-dnbi-dashboard-main/app-dnbi-dashboard/div/div[2]/div[1]/div[1]/app-widget-topten-company-list/div/div[2]/div[1]/div/button',
        )
    )
)


<selenium.webdriver.remote.webelement.WebElement (session="c6302b84fb1defdb781b3a9e163650d8", element="37C450CE76A17ADF2AC8E837FF43EA55_element_326")>

Now, the webscraping. Every company in the directory must be searched individually in Finance Analytics to get the data needed.

In [5]:
# Lists to store extracted data
company_names = []
addresses = []
phone_numbers = []
annual_sales = []
employees = []
line_of_businesses = []

# We will search every company on the directory.
# My strategy is that I will use advanced search to make the
# company details float on top of the search result
# as I experimented, this can be done by excluding unnecessary details
for i in range(len(Business_Directory_DF)):

    # Waiting for the page to properly load
    wait.until(
        condition.element_to_be_clickable(
            (
                By.XPATH,
                '//*[@id="main-body"]/dnbi-app/div/dnbi-app/dnbi-header/nav/div[2]/div[2]/div/div/span[1]/i',
            )
        )
    )

    # We will use the advance search filter in searching
    advance_filter = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/dnbi-app/div/dnbi-app/dnbi-header/nav/div[2]/div[2]/div/div/span[1]/i',
    )
    advance_filter.click()

    # Filling out the company name
    wait.until(
        condition.presence_of_element_located(
            (
                By.XPATH,
                '//*[@id="main-body"]/modal-container/div/div/advance-search/div[2]/tabset/div/tab[1]/div/div/form/div[1]/div[1]/div[3]/div[2]/ng-select/div/div/div[1]',
            )
        )
    )
    input_company = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/modal-container/div/div/advance-search/div[2]/tabset/div/tab[1]/div/div/form/div[1]/div[1]/div[2]/div[2]/input',
    )
    input_company.clear()
    input_company.send_keys(Business_Directory_DF.iloc[i]["company"])

    # Filling out the country
    country_dropdown = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/modal-container/div/div/advance-search/div[2]/tabset/div/tab[1]/div/div/form/div[1]/div[1]/div[3]/div[2]/ng-select/div/div/div[1]',
    )
    country_dropdown.click()
    input_country = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/modal-container/div/div/advance-search/div[2]/tabset/div/tab[1]/div/div/form/div[1]/div[1]/div[3]/div[2]/ng-select/select-dropdown/div/div[1]/input',
    )
    input_country.clear()
    input_country.send_keys(Business_Directory_DF.iloc[i]["country"])
    input_country.send_keys(Keys.ENTER)

    # Filling out the address by first 3 words only of the address
    # because the advanced search is too sensitive and I already did a lot of
    # experimentation and this step improved the accuracy of scraping
    input_address = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/modal-container/div/div/advance-search/div[2]/tabset/div/tab[1]/div/div/form/div[1]/div[1]/div[4]/div[2]/input',
    )
    input_address.clear()
    input_address.send_keys(Business_Directory_DF.iloc[i]["first3_address"])

    # Submitting the form
    submit = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/modal-container/div/div/advance-search/div[2]/tabset/div/tab[1]/div/div/form/div[3]/div[2]/button',
    )
    submit.click()

    # Waiting for the page to load
    wait.until(
        condition.presence_of_element_located(
            (
                By.XPATH,
                '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/nav/div/div[3]/div/div/div[3]/div[1]/div[1]/input',
            )
        )
    )
    active_business_only = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/nav/div/div[3]/div/div/div[3]/div[1]/div[1]/input',
    )

    # We are only after active businesses so we will filter active businesses
    active_business_only.click()

    # Click the dropdown preview to show the data we are looking for
    preview = driver.find_element(
        By.XPATH,
        '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[4]/div[1]/span[1]/span[1]',
    )
    preview.click()

    #  Wait for the preview to properly load
    wait.until(
        condition.presence_of_element_located(
            (
                By.XPATH,
                '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[4]/div[2]/div/div[1]/div[3]/span',
            )
        )
    )

    # Just leave blank space if we scraped no data
    company_name = ""
    address = ""
    phone_number = ""
    annual_sale = ""
    employee = ""
    line_of_business = ""

    # Checking if there's available data 

    # Scrape the company's name if it's available.
    # This is pretty redundant because it's already available on Business Directory DataFrame
    # but we will still include this one just for validation purposes
    try:
        company_name = driver.find_element(
            By.XPATH,
            '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[1]/div[1]/span[2]',
        )
        company_name = company_name.text
    except:
        NoSuchElementException

    # Scrape the company's address if it's available.
    # Also redundant because it's already available on Business Directory DataFrame
    # but we will still include this one because in my experience,
    # data from Finance Analytics are "more detectable" on Power BI Map
    try:
        address = driver.find_element(
            By.XPATH,
            '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[1]/div[3]/div[2]/span',
        )
        address = address.text
    except:
        NoSuchElementException

    # Scrape the company's phone number if it's available
    try:
        phone_number = driver.find_element(
            By.XPATH,
            '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[1]/div[4]/span[2]',
        )
        phone_number = phone_number.text
    except:
        NoSuchElementException

    # Scrape the company's annual sales if it's available
    try:
        annual_sale = driver.find_element(
            By.XPATH,
            '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[4]/div[2]/div/div[1]/div[1]/p',
        )
        annual_sale = annual_sale.text.replace("(Modelled)", "")
    except:
        NoSuchElementException

    # Scrape the company's number of employees if it's available
    try:
        employee = driver.find_element(
            By.XPATH,
            '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[4]/div[2]/div/div[1]/div[2]/p',
        )
        employee = employee.text
    except:
        NoSuchElementException

    # Scrape the company's line of business if it's available
    try:
        line_of_business = driver.find_element(
            By.XPATH,
            '//*[@id="main-body"]/dnbi-app/div/dnbi-app/div/dnbi-new-search-results/section[2]/div[2]/dnbi-search-result-entity[1]/div/div[4]/div[2]/div/div[1]/div[3]/p',
        )
        line_of_business = line_of_business.text
    except:
        NoSuchElementException

    # Append all the scraped data in their respective list
    company_names.append(company_name)
    addresses.append(address)
    phone_numbers.append(phone_number)
    annual_sales.append(annual_sale)
    employees.append(employee)
    line_of_businesses.append(line_of_business)


Putting the extracted data on a DataFrame

In [6]:
# Putting the list into dictionary
finance_analytics = {
    "FA_company": company_names,
    "FA_address": addresses,
    "FA_phone_number": phone_numbers,
    "FA_annual_sales": annual_sales,
    "FA_employees": employees,
    "FA_line_of_business": line_of_businesses,
}

# And then converting it into DataFrame
finance_analytics_DF = pd.DataFrame(finance_analytics)
finance_analytics_DF.head(5)


,FA_company,FA_address,FA_phone_number,FA_annual_sales,FA_employees,FA_line_of_business
0,"Shenzhen Angell Medical Equipment Co., Ltd.","West of 2/F, Building 5, Nanshui Industrial Vi...",+86 755 2681 4758,"US$ 10,728,750.00",13,Surgical and medical instruments
1,Jiangsu Huadong Medical Instrument Industrial ...,"No.88 Tongda Road, Touqiao Town, Hanjiang Dist...",+86 514 8748 5999,"US$ 10,728,750.00",80,Surgical and medical instruments
2,"Tianjin Zhite Medical Technology Co., Ltd.","East Zhang Gui Zhuang Railroad, Wai Huan Road,...",+86 22 2496 3247,"US$ 10,728,750.00",50,Surgical and medical instruments
3,"Ningbo Cibei Medical Applicances Co.,Ltd","No.8, Xiaotuanpu E. Road, Dong Development Zon...",+86 574 6366 7369,"US$ 10,728,750.00",73,Surgical and medical instruments
4,"Guilin Weirun Medical Technology Co., Ltd.","No.D-07, Informationa Industrial Park, High-Te...",54-820-6384,"US$ 10,728,750.00",37,Surgical and medical instruments


Consolidating business directory and finance analytics dataframe into a single DataFrame

In [7]:
consolidated_DF = pd.concat([Business_Directory_DF,finance_analytics_DF],axis=1)
consolidated_DF.head(5)

,company,directory_link,address,state,country,website,first3_address,FA_company,FA_address,FA_phone_number,FA_annual_sales,FA_employees,FA_line_of_business
0,"Shenzhen Angell Medical Equipment Co., Ltd.",https://www.dnb.com/business-directory/company...,"West of 2/F, Building 5, Nanshui Industrial Vi...",Guangdong,China,http://www.comermy.com.cn/,"West of 2/F,","Shenzhen Angell Medical Equipment Co., Ltd.","West of 2/F, Building 5, Nanshui Industrial Vi...",+86 755 2681 4758,"US$ 10,728,750.00",13,Surgical and medical instruments
1,Jiangsu Huadong Medical Instrument Industrial ...,https://www.dnb.com/business-directory/company...,"No.88 Tongda Road, Touqiao Town, Hanjiang Dist...",Yangzhou,China,http://www.yadamed.com/,"No.88 Tongda Road,",Jiangsu Huadong Medical Instrument Industrial ...,"No.88 Tongda Road, Touqiao Town, Hanjiang Dist...",+86 514 8748 5999,"US$ 10,728,750.00",80,Surgical and medical instruments
2,"Tianjin Zhite Medical Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"East Zhang Gui Zhuang Railroad, Wai Huan Road,...",Tianjin,China,NaN,East Zhang Gui,"Tianjin Zhite Medical Technology Co., Ltd.","East Zhang Gui Zhuang Railroad, Wai Huan Road,...",+86 22 2496 3247,"US$ 10,728,750.00",50,Surgical and medical instruments
3,"Ningbo Cibei Medical Applicances Co.,Ltd",https://www.dnb.com/business-directory/company...,"No.8, Xiaotuanpu E. Road, Dong Development Zon...",Cixi,China,http://www.cbzj.com.cn/,"No.8, Xiaotuanpu E.","Ningbo Cibei Medical Applicances Co.,Ltd","No.8, Xiaotuanpu E. Road, Dong Development Zon...",+86 574 6366 7369,"US$ 10,728,750.00",73,Surgical and medical instruments
4,"Guilin Weirun Medical Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"No.D-07, Informationa Industrial Park, High-Te...",Guilin,China,http://www.veirun.com/,"No.D-07, Informationa Industrial","Guilin Weirun Medical Technology Co., Ltd.","No.D-07, Informationa Industrial Park, High-Te...",54-820-6384,"US$ 10,728,750.00",37,Surgical and medical instruments


Exporting the consolidated DataFrame in existing file of business directory

In [8]:
book = load_workbook(directory_path)

with pd.ExcelWriter(directory_path, engine="openpyxl", mode="a") as writer:
    consolidated_DF.to_excel(writer, sheet_name="consolidated", index=False)